In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
# !pip install -U scikit-image==0.16.2


In [ ]:
import skimage
skimage.__version__

In [ ]:
# restructure the code for maskrcnn
!cp -r "/kaggle/input/object-detection-tf2/object-detection-master" "/kaggle/working/"
!cp "/kaggle/input/maskrcnnmodel/mask_rcnn_coco.h5" "/kaggle/working/object-detection-master/Mask_RCNN"
!cp -r "/kaggle/input/kangaroo-and-indian-busy-road/dataset" "/kaggle/working/object-detection-master/Mask_RCNN"

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0,"/kaggle/working/object-detection-master/Mask_RCNN")

In [ ]:
%cd /kaggle/working/object-detection-master/Mask_RCNN

In [ ]:
!python setup.py install

In [ ]:
# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from matplotlib import pyplot
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
import os
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image,ImageDraw
from mrcnn.utils import extract_bboxes
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn.utils import compute_ap
from matplotlib.patches import Rectangle


In [ ]:
# class that defines and loads the kangaroo dataset
class MathikereDataset(Dataset):
    
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # define one class
        self.add_class("dataset", 1, "person")
        self.add_class("dataset", 2, "twowheeler")
        self.add_class("dataset", 3, "fourwheeler")
        
        
        # define data locations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annotations/'
        
        # find all images
        for filename in listdir(images_dir):
            # extract image id
            image_id = filename[:-4]
            
            image_No = image_id.replace("frame_","")
                
            # skip all images after 150 if we are building the train set
            if is_train and int(image_No) <= 150:
                continue
                
            # skip all images before 150 if we are building the test/val set
            if not is_train and int(image_No) >= 150:
                continue
                
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # add to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    # load the masks for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location
        path = info['annotation']
        # load XML
        _,masks,_,class_ids, _, _ = self.extract_helper(path)            
        return masks, asarray(class_ids, dtype='int32')

    # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']
    

    # extract bounding boxes from an annotation file
    def extract_helper(self,filename):

        # labels

        # load and parse the file
#         path = "/home/saurav/Documents/Learning/KangarooDetection/Mask_RCNN/dataset/mathikere"
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()

        width = int(root.find('.//imagesize/nrows').text)
        height = int(root.find('.//imagesize/ncols').text)

#         print(width,height)

        boxes = list()
        names = list()
        img_polygons = list()
        masks = list()
        class_ids = list()

        for member in root.findall('object'):
            polygons = list()
            name = member.find("name").text
            class_id = self.class_names.index(name)

            for polypoints in member.findall('.//polygon/pt'):            
                x = int(float(polypoints.find("x").text))
                y = int(float(polypoints.find("y").text))
                polygons.append((x,y))

            # # polygons to bbox
            x,y,w,h = cv2.boundingRect(np.array(polygons))
            boxes.append((x,y,x+w,y+h))

            # polypoints to masks
            img = Image.new('L', (height,width), 0)
            ImageDraw.Draw(img).polygon(polygons, outline=1, fill=1)
            mask = np.array(img)


            # Add to list
            img_polygons.append(polygons)
            names.append(name)
            masks.append(mask)
            class_ids.append(class_id)
            
        masks = np.transpose(np.array(masks), (1, 2,0))


        return np.array(img_polygons),np.array(masks), np.array(boxes), np.array(class_ids), width, height

    
    

In [ ]:
# train set
train_set = MathikereDataset()
train_set.load_dataset('./dataset/mathikere', is_train=True)
train_set.prepare()
# load an image
image_id = 0
image = train_set.load_image(image_id)
print(image.shape)
# load image mask
mask, class_ids = train_set.load_mask(image_id)
print(mask.shape)
# plot image
pyplot.imshow(image)
# plot mask
pyplot.imshow(mask[:, :, 0], cmap='gray', alpha=0.5)
pyplot.show()

In [ ]:
# train set
train_set = MathikereDataset()
train_set.load_dataset('./dataset/mathikere', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# test/val set
test_set = MathikereDataset()
test_set.load_dataset('./dataset/mathikere', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [ ]:
# train set
train_set = MathikereDataset()
train_set.load_dataset('./dataset/mathikere', is_train=True)
train_set.prepare()
# load an image
image_id = 0
image = train_set.load_image(image_id)
print(image.shape)
# load image mask
mask, class_ids = train_set.load_mask(image_id)
print(mask.shape)
# plot image
# pyplot.imshow(image)
# plot mask
pyplot.imshow(mask[:, :, 0], cmap='gray', alpha=0.5)
pyplot.show()

In [ ]:
def show_many():
    # plot first few images
    for i in range(9):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # plot raw pixel data
        image = train_set.load_image(i)
        pyplot.imshow(image)
        # plot all masks
        mask, _ = train_set.load_mask(i)
        for j in range(mask.shape[2]):
            pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
    # show the figure
    pyplot.show()
    
show_many()

In [ ]:
# using mrcnn visualise
# define image id
image_id = 1
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)

In [ ]:
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)

In [ ]:
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

In [ ]:
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

In [ ]:
# define a configuration for the model
class MathikereTrainConfig(Config):
    # define the name of the configuration
    NAME = "kangaroo_cfg"
    # number of classes (background + no of class)
    NUM_CLASSES = 1 + 3
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    # number of training steps per epoch
    STEPS_PER_EPOCH = 131

In [ ]:
# train set
train_set = MathikereDataset()
train_set.load_dataset('./dataset/mathikere', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

In [ ]:
# test/val set
test_set = MathikereDataset()
test_set.load_dataset('./dataset/mathikere', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

In [ ]:
# prepare config
config = MathikereTrainConfig()
config.display()

In [ ]:
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')

In [ ]:
# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "kangaroo_cfg"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 1 + 3
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
    APs = list()
    for image_id in dataset.image_ids:
        # load image, bounding boxes and masks for the image id
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id)
        # convert pixel values (e.g. center)
        scaled_image = mold_image(image, cfg)
        # convert image into one sample
        sample = np.expand_dims(scaled_image, 0)
        # make prediction
        yhat = model.detect(sample, verbose=0)
        # extract results for first sample
        r = yhat[0]
        # calculate statistics, including AP
        AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
        # store
        APs.append(AP)
    # calculate the mean AP across all images
    mAP = mean(APs)
    return mAP


# create config
cfg = PredictionConfig()

In [ ]:
!ls -l -1t

In [ ]:
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)

# load model weights
# mask_rcnn_kangaroo_cfg_{epoch:04d}.h5
model.load_weights('kangaroo_cfg20210827T1634/mask_rcnn_kangaroo_cfg_0005.h5', by_name=True)

In [ ]:
# evaluate model on training dataset
# FAILED: ValueError: shapes (100,1048576) and (3136,22) not aligned: 1048576 (dim 1) != 3136 (dim 0)
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)

In [ ]:
# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, n_images=5):
    # load image and mask
    for i in range(n_images):
        # load the image and mask
        image = dataset.load_image(i)
        mask, _ = dataset.load_mask(i)
        # convert pixel values (e.g. center)
        scaled_image = mold_image(image, cfg)
        # convert image into one sample
        sample = np.expand_dims(scaled_image, 0)
        # make prediction
        yhat = model.detect(sample, verbose=0)[0]
        # define subplot
        pyplot.subplot(n_images, 2, i*2+1)
        # plot raw pixel data
        pyplot.imshow(image)
        pyplot.title('Actual')
        # plot masks
        for j in range(mask.shape[2]):
            pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
        # get the context for drawing boxes
        pyplot.subplot(n_images, 2, i*2+2)
        # plot raw pixel data
        pyplot.imshow(image)
        pyplot.title('Predicted')
        ax = pyplot.gca()
        # plot each box
        for box in yhat['rois']:
            # get coordinates
            y1, x1, y2, x2 = box
            # calculate width and height of the box
            width, height = x2 - x1, y2 - y1
            # create the shape
            rect = Rectangle((x1, y1), width, height, fill=False, color='red')
            # draw the box
            ax.add_patch(rect)
    # show the figure
    pyplot.show()

In [ ]:
# plot predictions for train dataset
plot_actual_vs_predicted(train_set, model, cfg)

# plot predictions for test dataset
plot_actual_vs_predicted(test_set, model, cfg)

In [ ]:
!zip ../model.zip kangaroo_cfg20210827T1634/mask_rcnn_kangaroo_cfg_0005.h5

<a href="./object-detection-master/model.zip"> Download File </a>